In [1]:
import model_based_mvpa as mbmvpa
from model_based_mvpa.preprocessing.bids import *
from model_based_mvpa.preprocessing.events import *
from model_based_mvpa.utils.functions import *

In [2]:
root = '/data2/project_model_based_fmri/ds001882/'

In [3]:
X, masked_data, layout = bids_preprocess(root, smoothing_fwhm=None, zoom=(2, 2, 2))

image preprocessing - fMRI data 2 / 7..           :  67%|██████▋   | 4/6 [05:26<00:28, 14.17s/it]/home/mybirth0407/anaconda3/envs/mvpa/lib/python3.7/site-packages/scipy/ndimage/interpolation.py:478: UserWarning: The behaviour of affine_transform with a one-dimensional array supplied for the matrix parameter has changed in scipy 0.18.0.
  "The behaviour of affine_transform with a one-dimensional "
/home/mybirth0407/anaconda3/envs/mvpa/lib/python3.7/site-packages/scipy/ndimage/interpolation.py:478: UserWarning: The behaviour of affine_transform with a one-dimensional array supplied for the matrix parameter has changed in scipy 0.18.0.
  "The behaviour of affine_transform with a one-dimensional "
/home/mybirth0407/anaconda3/envs/mvpa/lib/python3.7/site-packages/scipy/ndimage/interpolation.py:478: UserWarning: The behaviour of affine_transform with a one-dimensional array supplied for the matrix parameter has changed in scipy 0.18.0.
  "The behaviour of affine_transform with a one-dimensio

In [4]:
import os
from pathlib import Path
import bids
from bids import BIDSLayout, BIDSValidator

import pandas as pd
import numpy as np
from sklearn.preprocessing import minmax_scale

from nilearn.glm.first_level.hemodynamic_models import compute_regressor
import nibabel as nib
from tqdm import tqdm

import hbayesdm.models
import time

import logging

layout = BIDSLayout(root, derivatives=True)
t_r = layout.get_tr()
events = layout.get(suffix='events', extension='tsv')
image_sample = nib.load(
    layout.derivatives['fMRIPrep'].get(
        return_type='file',
        suffix='bold',
        extension='nii.gz')[0]
)
n_scans = image_sample.shape[-1]
df_events = pd.read_table('dd_hyperbolic_latent.tsv')
frame_times = t_r * (np.arange(n_scans) + t_r/2)
hrf_model='glover'

In [5]:
print(t_r, n_scans, frame_times.shape)

2.0 219 (219,)


In [6]:
signals = []
for name0, group0 in df_events.groupby(['subjID']):
    signal_subject = []
    for name1, group1 in group0.groupby(['run']):
        exp_condition = group1[['onset', 'duration', 'modulation']].to_numpy().T

        signal, name = compute_regressor(
            exp_condition=exp_condition,
            hrf_model=hrf_model,
            frame_times=frame_times)
        signal_subject.append(signal)

    signal_subject = np.array(signal_subject)
    reshape_target = signal_subject.shape

    normalized_signal = minmax_scale(signal_subject.flatten(), axis=0, copy=True)
    normalized_signal = normalized_signal.reshape(-1, n_scans, 1)
    signals.append(normalized_signal)
signals = np.array(signals)

In [7]:
np.save('/data2/project_model_based_fmri/ds001882/derivatives/fmriprep/data/y.npy', signals)

In [8]:
y = signals

## model

In [9]:
from model_based_mvpa.models.regressor import *
from model_based_mvpa.utils.coef2map import *
import time

In [ ]:
X, y = prepare_data(X=root + 'derivatives/fmriprep/data/X.npy',
                    y=root + 'derivatives/fmriprep/data/y.npy')
masked_data = nib.load(root + 'derivatives/fmriprep/data/masked_data.nii.gz')

In [10]:
coefs = mlp_regression(X, y,
                       layer_dims=[1024, 1024],
                       activation_func='linear',
                       dropout_rate=0.5,
                       epochs=100,
                       patience=10,
                       batch_size=64,
                       N=15,
                       verbose=1)

INFO:root:start running


ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type numpy.ndarray).

In [ ]:
task_name = 'tom2007_mlp'

In [ ]:
result = get_map(coefs, masked_data, task_name, map_type='z', save_path=None, smoothing_sigma=1)

In [ ]:
print(time.strftime('%c', time.localtime(time.time())))

In [ ]:
coefs = penalized_linear_regression(X, y,
                         alpha=0.001, 
                         lambda_par=0.8,
                         epochs=100,
                         patience=30,
                         batch_size=256,
                         N=15,
                         verbose=1)

In [ ]:
task_name = 'tom2007_penlin'

In [ ]:
result = get_map(coefs, masked_data, task_name, map_type='z', save_path=None, smoothing_sigma=1)

In [ ]:
print(time.strftime('%c', time.localtime(time.time())))

In [ ]:
coefs = elasticnet(X, y,
             alpha=0.001,
             n_jobs=16,
             N=3,
             verbose=1)

In [ ]:
task_name = 'tom2007_elasticnet'

In [ ]:
result = get_map(coefs, masked_data, task_name, map_type='z', save_path=None, smoothing_sigma=1)

In [ ]:
print(time.strftime('%c', time.localtime(time.time())))